MLP to train CIFAR-10 Multi-class classification

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn

import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as td
import random, time
import torchvision

print('lib import succeed')
torch.cuda.empty_cache()

load CIFAR-10 dataset

In [ ]:

def cifar_loaders(batch_size, shuffle_test=False): 
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.225, 0.225, 0.225])
    train = datasets.CIFAR10('./', train=True, download=True, 
        transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]))
    test = datasets.CIFAR10('./', train=False, 
        transform=transforms.Compose([transforms.ToTensor(), normalize]))
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
        shuffle=True, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size,
        shuffle=shuffle_test, pin_memory=True)
    return train_loader, test_loader

batch_size = 64
test_batch_size = 64

train_loader, _ = cifar_loaders(batch_size)
_, test_loader = cifar_loaders(test_batch_size)

check what we downloaded:

CIFAR-10 contains 50000 training images and 10000 training images

50000/batch size = len(train_loader) 

10000/batch size = len(test_loader)

per image: 32x32 with 3 channels

In [ ]:
print(f'training dataset size: {len(train_loader)}')

i = 0;
for what in train_loader:
    for image in what[0]:
        print(len(image))
    for class_id in what[1]:
        print(class_id)
    
    # print(whaxt)
    i = i + 1
    if(i > 0):
        break;

# print(f'per image size: {len(train_loader[0])}')
print(f'testing dataset size: {len(test_loader)}')

define some parameters

In [ ]:
learning_rate = 1e-2
layer_no = 7
w_relu = False

define neural network here - MLP (fully connected)

In [ ]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super(MLP, self).__init__() 

        self.lay_no = layer_no
        self.input_size = 3 * 32 * 32
        self.output_size = 10
        self.per_layer_diff = round(3 * 32 * 32 / layer_no, 0)
        
        self.fc_layers = nn.ModuleList()

        print(self.per_layer_diff)
        print(self.input_size - (1-1) * self.per_layer_diff)
        print(self.input_size - 1 * self.per_layer_diff)

        # Corrected dimensions for fcl1 and fcl2
        for i in range(self.lay_no):
            in_dim = int(self.input_size - (i + 1 -1) * self.per_layer_diff)
            out_dim = int(self.input_size - (i+1) * self.per_layer_diff) if i < layer_no - 1 else self.output_size

            layer = nn.Linear(in_dim, out_dim, bias=True)
            self.fc_layers.append(layer)
            
        for what in self.fc_layers:
            print(what)
            
        print("MLP INITIALIZED!")

    
    def forward(self, x):
        x = x.view(-1, self.input_size)
        
        for layer in self.fc_layers:
            x = layer(x)
            if w_relu:
                x = F.relu(x)  # Applying ReLU activation after each layer

        return x
        
        
        
        
net = MLP()

more predefinition before training

In [ ]:
epoch_no = 20
classnames = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
print(net.fc_layers)
print(device)

defind loss function and optimizer

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4) # weight_decay for regularization

start training

In [ ]:
import os 

begin_time = time.time()
logname = 'result_MLP.txt'

if os.path.exists(logname):
    os.remove(logname)
f = open(logname, 'w')
f.write('Run Start Time: ' + str(time.ctime()))
f.write('Learning Rate\t%f\n' % learning_rate)

for epoch in range(epoch_no):
    epoch_start_time = time.time()
    epoch_loss = 0.0
    
    # train
    for i, data in enumerate(train_loader):
        
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net.forward(images)
        batch_loss = loss_func(outputs, labels)
        
        batch_loss.backward()
        optimizer.step()
        
        epoch_loss += batch_loss.item()
        
    f.write("Epoch\t%d\tLoss\t%f\t" % (epoch + 1, epoch_loss / (i + 1)))
    epoch_end_time = time.time()
    
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    correct = 0
    total = 0
    
    # test
    with torch.no_grad():
        for data in test_loader:
            
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            
            outputs = net.forward(images)
            _, predicted = torch.max(outputs.data, 1)
            correct_matrix = (predicted == labels)

            c = correct_matrix.squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
            total += labels.size(0)
            correct += correct_matrix.sum().item()
            
    f.write('Accuracy of the network [%d/%d]\t%f %%\n' % (correct, total, 100 * correct / total))
    print(f'Epoch:{epoch} Complete!')

f.write('Training Complete: ' + str(time.ctime()) + '\n')
run_time = time.time() - begin_time
f.write('Runtime\t%f\n' % run_time)
f.close()




In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np

namehere = 'result_MLP'
# Open the file and read its content
with open(namehere+'.txt', 'r') as file:
    content = file.read()

# Use regular expressions to find loss and accuracy values
loss_pattern = re.compile(r'Epoch\s*\d+\s*Loss\s*([\d.]+)')
accuracy_pattern = re.compile(r'Accuracy of the network \[([\d]+)/\d+\]\s*([\d.]+)')

# Find all occurrences of loss and accuracy in the content
loss_values = [float(match.group(1)) for match in loss_pattern.finditer(content)]
accuracy_values = [float(match.group(2)) for match in accuracy_pattern.finditer(content)]

# Print the extracted values
print("Loss values:", loss_values)
print("Accuracy values:", accuracy_values)

In [ ]:


filename = namehere + ".pdf"

plt.rcParams["figure.figsize"] = [8.0,4.0]
plt.rcParams["figure.autolayout"] = True
plt.title(namehere)

x = list(range(1,21)) 
plt.plot(x,loss_values, '-bD',  c='blue', mfc='red', mec='k')
plt.xticks(np.arange(min(x), max(x)+1, 1.0))
plt.xlabel("epochs", )
plt.ylabel("Loss/No. of Batch")
plt.ylim(0.0,2.5)
plt.savefig("./" + filename, format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
filename = namehere + "_accuracy" + ".pdf"

plt.rcParams["figure.figsize"] = [8.0,4.0]
plt.rcParams["figure.autolayout"] = True
plt.title("Accuracy @ " + namehere)

x = list(range(1,21))
plt.plot(x, accuracy_values, '-bD',  c='blue', mfc='red', mec='k')
plt.xticks(np.arange(min(x), max(x)+1, 1.0))
plt.xlabel("epochs", )
plt.ylabel("Accuracy (%)")
plt.ylim(20, 90)
plt.savefig("./" + filename, format="pdf", bbox_inches="tight")
plt.show()